In [1]:
!pip install open-flamingo
!pip install pycocoevalcap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# grab model checkpoint from huggingface hub
import sys
sys.path.insert(0, os.path.abspath('..'))
import torch
import json
from PIL import Image
from tqdm import tqdm
from textwrap import wrap
from evaluation.evaluation import eval
from huggingface_hub import hf_hub_download
from open_flamingo import create_model_and_transforms

In [3]:


model, image_processor, tokenizer = create_model_and_transforms(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b",
    tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b",
    cross_attn_every_n_layers=1
)

100%|███████████████████████████████████████| 933M/933M [00:17<00:00, 53.4MiB/s]


Using pad_token, but it is not set yet.


A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- configuration_mosaic_gpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- low_precision_layernorm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- attention.py
- low_precision_layernorm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- gpt_blocks.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b:
- mosaic_gpt.py
- attention.py
- param_init_fns.py
- gpt_blocks.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50280. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Flamingo model initialized with 1046992944 trainable parameters


In [4]:


checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

_IncompatibleKeys(missing_keys=['vision_encoder.class_embedding', 'vision_encoder.positional_embedding', 'vision_encoder.proj', 'vision_encoder.conv1.weight', 'vision_encoder.ln_pre.weight', 'vision_encoder.ln_pre.bias', 'vision_encoder.transformer.resblocks.0.ln_1.weight', 'vision_encoder.transformer.resblocks.0.ln_1.bias', 'vision_encoder.transformer.resblocks.0.attn.in_proj_weight', 'vision_encoder.transformer.resblocks.0.attn.in_proj_bias', 'vision_encoder.transformer.resblocks.0.attn.out_proj.weight', 'vision_encoder.transformer.resblocks.0.attn.out_proj.bias', 'vision_encoder.transformer.resblocks.0.ln_2.weight', 'vision_encoder.transformer.resblocks.0.ln_2.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_fc.bias', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.weight', 'vision_encoder.transformer.resblocks.0.mlp.c_proj.bias', 'vision_encoder.transformer.resblocks.1.ln_1.weight', 'vision_encoder.transformer.resbloc

In [5]:

# Open the captions_train2017.json file
with open('/kaggle/input/coco-2017-dataset/coco2017/annotations/captions_val2017.json', 'r') as f:
    data = json.load(f)
    data = data['annotations']
# Check the number of captions
id_list = list(set([d['image_id'] for d in data]))

In [6]:
def get_result(prompt=''):
    result = []
    for i in tqdm(id_list[:500]):
        image_token = "<image>"
        end_token = "<|endofchunk|>"
        img_name = '%012d.jpg' % i
        path = f'/kaggle/input/coco-2017-dataset/coco2017/val2017/{img_name}'
        query_image = Image.open(path)
        
        vision_x = [image_processor(query_image).unsqueeze(0)]
        vision_x = torch.cat(vision_x, dim=0)
        vision_x = vision_x.unsqueeze(1).unsqueeze(0)
        tokenizer.padding_side = "left"
        lang_x = tokenizer(
            [f"{image_token}{prompt}"],
            return_tensors="pt",
        )
        
        generated_text = model.generate(
            vision_x=vision_x,
            lang_x=lang_x["input_ids"],
            attention_mask=lang_x["attention_mask"],
            max_new_tokens=20,
        )
        start_idx = len(image_token)+len(prompt)
        end_idx = len(end_token)
        caption = tokenizer.decode(generated_text[0])
        result.append({"image_id":i, "caption":caption[start_idx:-end_idx]})
    return result

In [7]:
def get_one_shot_result(prompt=''):
    result = []
    image_token = "<image>"
    end_token = "<|endofchunk|>"
    tokenizer.padding_side = "left"
    prompt_image_path = '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000266677.jpg'
    prompt_image = Image.open(prompt_image_path)
    prompt_image_x = image_processor(prompt_image).unsqueeze(0)
    prompt_image_caption = 'A woman holding a microphone stands in front of two giraffes.'
    prompt = f"{image_token}An image of {prompt_image_caption}{end_token}{image_token}An image of"
    lang_x = tokenizer(
        [prompt],
        return_tensors="pt",
    )

    for i in tqdm(id_list[:500]):
        img_name = '%012d.jpg' % i
        path = f'/kaggle/input/coco-2017-dataset/coco2017/val2017/{img_name}'
        query_image = Image.open(path)
        vision_x = [prompt_image_x, image_processor(query_image).unsqueeze(0)]
        vision_x = torch.cat(vision_x, dim=0)
        vision_x = vision_x.unsqueeze(1).unsqueeze(0)
        generated_text = model.generate(
            vision_x=vision_x,
            lang_x=lang_x["input_ids"],
            attention_mask=lang_x["attention_mask"],
            max_new_tokens=20,
        )
        caption = tokenizer.decode(generated_text[0])
        result.append({"image_id":i, "caption":caption[len(prompt):-len(end_token)]})
    return result

In [8]:
result_zero_shot = get_result()

100%|██████████| 500/500 [2:02:04<00:00, 14.65s/it]


In [9]:
result_one_shot = get_one_shot_result()

100%|██████████| 500/500 [3:13:19<00:00, 23.20s/it]


In [11]:
# save as a json
with open('zero_shot_results.json', 'w') as json_file:
    json.dump(result_zero_shot, json_file)# save as a json
with open('one_shot_results.json', 'w') as json_file:
    json.dump(result_one_shot, json_file)

In [12]:
eval('zero_shot_results.json')

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 30607 tokens at 95308.24 tokens per second.
PTBTokenizer tokenized 3918 tokens at 24080.82 tokens per second.


setting up scorers...
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
computing Bleu score...
{'testlen': 3138, 'reflen': 4552, 'guess': [3138, 2638, 2151, 1709], 'correct': [1354, 382, 104, 37]}
ratio: 0.6893673110719047
Bleu_1: 0.275
Bleu_2: 0.159
Bleu_3: 0.092
Bleu_4: 0.057
computing METEOR score...
METEOR: 0.113
computing Rouge score...
ROUGE_L: 0.241
computing CIDEr score...
CIDEr: 0.315
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [1.0 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.9 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.7 sec].
Loading classif

SPICE evaluation took: 1.301 min
SPICE: 0.084
Bleu_1: 0.275
Bleu_2: 0.159
Bleu_3: 0.092
Bleu_4: 0.057
METEOR: 0.113
ROUGE_L: 0.241
CIDEr: 0.315
SPICE: 0.084


In [13]:
eval('one_shot_results.json')

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 30607 tokens at 144273.92 tokens per second.
PTBTokenizer tokenized 4636 tokens at 36086.33 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 3587, 'reflen': 4464, 'guess': [3587, 3087, 2587, 2093], 'correct': [2643, 1280, 492, 187]}
ratio: 0.8035394265231174
Bleu_1: 0.577
Bleu_2: 0.433
Bleu_3: 0.303
Bleu_4: 0.210
computing METEOR score...
METEOR: 0.212
computing Rouge score...
ROUGE_L: 0.481
computing CIDEr score...
CIDEr: 0.827
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [1.2 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.0 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.9

SPICE evaluation took: 25.73 s
SPICE: 0.160
Bleu_1: 0.577
Bleu_2: 0.433
Bleu_3: 0.303
Bleu_4: 0.210
METEOR: 0.212
ROUGE_L: 0.481
CIDEr: 0.827
SPICE: 0.160
